In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
X, y = load_iris(return_X_y=True)
x_train, x_val, y_train, y_val = train_test_split(X, y, random_state=0)

import os 
path=os.chdir('c:\\Users\\constantinoj\\Desktop\\Prueba_keras_tune')

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler  # Added for data normalization
from scikeras.wrappers import KerasClassifier

In [2]:
pwd

'c:\\Users\\constantinoj\\Desktop\\Prueba_keras_tune'

In [3]:
print(np.unique(y_train))
print(np.unique(y_val))

[0 1 2]
[0 1 2]


In [29]:
#modelo1=keras.Sequential()
for i in [1,2,3]:
    print(i)

1
2
3


In [30]:
def build_modelo(hp):
    modelo = keras.Sequential()
        # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 9)):
        modelo.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    
    if hp.Boolean("dropout"):
        modelo.add(layers.Dropout(rate=0.25))
    
    modelo.add(layers.Dense(3, activation="softmax"))

    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    modelo.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return modelo

tuner = RandomSearch(
    build_modelo,
    objective='val_accuracy',# 
    max_trials=7,
    executions_per_trial=4,
    overwrite=True,
    directory=path,
    project_name="Primer_acercamiento",)

tuner.search(x_train, y_train,
             epochs=5,# numero de modelos 
             validation_data=(x_val, y_val))

Trial 7 Complete [00h 00m 28s]
val_accuracy: 0.8552631735801697

Best val_accuracy So Far: 0.9605263024568558
Total elapsed time: 00h 02m 56s


In [5]:
tuner.directory

'.'

In [16]:
type(path)

NoneType

In [17]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

c:\Users\constantinoj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [31]:
# Ejercicio de muestra como busca en las diferentes arquitecturas hasta quedarse con la mas eficiente 
for i in range(len(tuner.get_best_models(num_models=4))):
    tuner.get_best_models(num_models=4)[i].summary()

c:\Users\constantinoj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\constantinoj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\constantinoj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
c:\Users\constantinoj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │        16,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,243 (86.89 KB)

 Trainable params: 22,243 (86.89 KB)

...

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 192)            │        49,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 288)            │        55,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 192)            │        24,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 177,411 (693.01 KB)

 Trainable params: 177,411 (693.01 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 320)            │         1,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        82,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 352)            │        90,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │        11,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 185,635 (725.14 KB)

 Trainable params: 185,635 (725.14 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# De las descripciones largas se obtiene el histograma para la desición del tamaño inical del embeding 
NUM_PALABRAS=2000


# construimos el modelo
def build_model_clasif_gen(hp4):
    # El parametro (hp4 )que va a crear un diccionario de los numeros de 
    # Iniciamos la red neuronal sin capas 
    modelo=keras.Sequential()
    # Agregamos la capa de enbedding
    modelo.add(layers.Embedding(NUM_PALABRAS,output_dim=hp4.Int("units",min_value=60, max_value=300,step=60)))
    # Desactivamos un porcentaje de las 
    modelo.add(layers.Dropout(0.1))
    # Aplanamos la estructura en un vector largo 
    modelo.add(layers.Flatten())
    # Agregamos una capa totalmente conectada 
    modelo.add(layers.Dense(30,activation="relu"))
    # aplicamos capa de desactivacion (porcentaje )
    modelo.add(layers.Dropout(0.1))
    # Agregamos una capa totalmente conectada 
    modelo.add(layers.Dense(12, activation="relu"))
    # Agregamos capa de olvido
    modelo.add(layers.Dropout(0.25))
    # Agregamos capa de salida
    modelo.add(layers.Dense(3, activation='softmax'))
    
    # Programamos los hiperparametros 
    learning_rate = hp4.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")

    # compliacion del modelo
    modelo.compile(
        # Importamos el Optimizador de adam ( con el buscador de hiperparametros)
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy", # Perdida asociada a la forma de los 
        metrics=["accuracy"]) # Precicion de la clasificacion 
    return modelo

# creamos el tipo de busqueda aleatoria



tuner1 = RandomSearch(
    build_model_clasif_gen,# pasamos la arquitectura
    objective='val_accuracy',# me damos el metodo al buscador de Hiperparametros 
    max_trials=5,# maximo de intentos
    executions_per_trial=6,# numero de intentos por iteración 
    overwrite=False, # Sobre escritura 
    directory=path, # Direccion donde se almacenaran los resultados
    project_name=path)# Nombre del proyecto 

In [22]:
tuner1.search(x_train, #Variables predictoras
              y_train,# Variables predictoras 
             epochs=2,# numero de epocas ( Este parametro se optimizara mas adelante )
             validation_data=(x_val, y_val))

Trial 5 Complete [00h 00m 18s]
val_accuracy: 0.8815789421399435

Best val_accuracy So Far: 0.8815789421399435
Total elapsed time: 00h 01m 27s


In [24]:
# validamos el lugar donde esta el modelo
tuner.directory

'.'

In [26]:
# Nos quedamos con el modelo que mejoro con mejor precicion 
best_model = tuner1.get_best_models(num_models=1)[0]

c:\Users\constantinoj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [28]:
# validamos la arquitectura encontrada por keras-tuner
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 4, 240)         │        24,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4, 240)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 960)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 30)             │        28,830 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │           372 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            39 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,241 (207.97 KB)

 Trainable params: 53,241 (207.97 KB)

 Non-trainable params: 0 (0.00 B)